## N-Body Differentiable PP Solver — Inverse Problem Sampling with HMC/NUTS

This pipeline simulates gravitational interactions between particles forming one or two dark matter halos, using physically grounded initial conditions and a symplectic integrator. It is implemented in JAX using the `diffrax` solver and leverages CIC painting for density field output.

---

## 1. Physical Model

### 1.1 Newtonian Gravity with Softening

- **Force law:**
  $$ \vec{F}_{ij} = -G \frac{m_i m_j (\vec{r}_i - \vec{r}_j)}{(|\vec{r}_i - \vec{r}_j|^2 + \epsilon^2)^{3/2}}$$
- **Softening** avoids divergence at short range.
- **Periodic boundary conditions** are implemented using the **minimum-image convention**:
  $$
  \Delta \vec{r}_{ij} \mapsto \Delta \vec{r}_{ij} - L \cdot \text{round}\left(\frac{\Delta \vec{r}_{ij}}{L}\right)
  $$

### 1.2 Time Evolution: Leapfrog Integration

- Uses the **LeapfrogMidpoint** integrator from `diffrax`, a symplectic method:
  - Preserves phase-space volume and total energy better over long times.
  - Suitable for Hamiltonian systems like gravitational N-body problems.

---

## 2. Initial Conditions

### 2.1 Gaussian Spatial Distribution

- Positions are sampled from a 3D isotropic Gaussian:
  $$
  \vec{x}_i \sim \mathcal{N}(\vec{\mu}, \sigma^2 I)
  $$

### 2.2 Velocity Initialization

- If `random_vel=True`: adds Gaussian random velocities ($\sigma_v$).
- In all cases: adds **circular velocity** computed from enclosed mass.

#### Enclosed Mass for Gaussian Profile

- Analytical enclosed mass formula:
  $$
  M(<r) = M \left[ \operatorname{erf}\left(\frac{r}{\sqrt{2}\sigma}\right) - \sqrt{\frac{2}{\pi}} \frac{r}{\sigma} e^{-r^2 / (2\sigma^2)} \right]
  $$

#### Circular Velocity

- Velocity magnitude:
  $$
  v_c(r) = \sqrt{\frac{G M(<r)}{r}}
  $$
- Direction: orthogonal to radius vector in XY-plane via cross-product with the Z-axis.

---

## 3. Density Field Output

- Uses **CIC (Cloud-in-Cell)** from `jaxpm.painting.cic_paint` to map particles to a 3D grid:
  $$
  \rho(\vec{x}) = \sum_i W(\vec{x} - \vec{x}_i)
  $$

---

## 4. Code Structure

| Function | Purpose |
|---------|---------|
| `pairwise_forces` | Computes gravitational forces with softening and periodic BCs. |
| `make_diffrax_ode` | Returns an ODE system for use with `diffrax`. |
| `blob_circular_velocity_gaussian` | Computes circular velocity from enclosed mass. |
| `gaussian_model` | Runs simulation for one Gaussian halo. |
| `gaussian_2blobs` | Runs simulation for two interacting halos. |

---

## 5. Potential Improvements

| Issue | Comment |
|-------|---------|
| **Mass scaling in force calculation** | Force uses \( r^{-3} \) but divides acceleration by `m_part`; ensure consistency. |
| **Velocity keys** | Both blobs use same key for velocity → can lead to correlations. Use separate PRNG splits. |
| **No bulk motion** | Two-blob model lacks explicit center-of-mass motion; could simulate merging more realistically. |
| **Fixed rotation axis** | Circular velocity lies in XY-plane. A general angular momentum initialization may be better. |
| **Energy diagnostics** | Tracking total energy would help validate numerical stability. |
| **Add cosmological expansion** | Add cosmological expansion (scale factor, comoving coordinates).|
| **Track energy** | Track energy, angular momentum over time.|
| **Alternative distribution** | Support alternative profiles: NFW, Plummer, Hernquist.|

## Inverse problem inference

 We call $f_{t}(\theta)$  the density field of Dark Matter at time $t$ which can be paramterised by $\theta$ at initial time. 

 We have a forward model that maps, $f_{0}(\theta)$ with $f_{t_f}(\theta)$, by simulating the dynamic of samples of particles from $f_{0}(\theta)$ between $0$ and $t_f$. 



 We then suppose to have only access to the final density field with some gaussian noise : $$\textbf{data} = f_{t_f}(\theta) + \epsilon.$$

 If our output density field is discretized by some $N$ points $y_i$, this gives for our noised observed data : 
 
$$z_i = y_i(\theta) + \epsilon_i,\quad \epsilon_i \sim \mathcal{N}(0, \sigma^2)$$

Then the likelihood for our observations is :

$$p(\mathbf{z} \mid \theta) = \prod_{i=1}^N \frac{1}{\sqrt{2\pi \sigma^2}} 
\exp\left( - \frac{1}{2\sigma^2} (z_i - y_i(\theta))^2 \right)$$

So finally our log-likelihood for our data is : 
$$\log \mathcal{L}(\theta) = -\frac{1}{2\sigma^2} 
\sum_{i=1}^N (y_i - f_i(\theta))^2 - 
\frac{N}{2}\log(2\pi \sigma^2)$$
